In [6]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import csv

# Initialize a Selenium web driver
driver = webdriver.Chrome()
base_url = 'https://tain.tw/stats?type=2&points_page=1'
driver.get(base_url)

# Wait for the page to load (you can adjust the sleep time)
time.sleep(3)

# Create an empty list to store all the data
all_data = []

# Define the desired column names
column_names = ['Pos', '球員', '排名', '勝率', '直板勝率', '橫板勝率', '右手勝率', '左手勝率', '男子勝率', '女子勝率', '穩定率', '爆發率', '平面勝率', '短顆勝率', '中顆勝率', '長顆勝率', 'NT勝率']

# Add the column names as the first row
all_data.append(column_names)

for i in range(0, 144):
    # Find the table using XPath
    table = driver.find_element(By.XPATH, "//table[@class='table table-sm table-hover text-nowrap fs-caption']")

    # Get the HTML content of the table
    table_html = table.get_attribute('outerHTML')

    # Parse the HTML content using BeautifulSoup
    soup_table = soup(table_html, 'html.parser')

    # Find all table rows
    rows = soup_table.find_all('tr')

    # Create empty lists to store the data for the current page
    data = []

    # Iterate through the rows and extract data
    for row in rows[1:]:  # Skip the header row
        columns = row.find_all(['th', 'td'])
        row_data = [column.get_text(strip=True) for column in columns]
        data.append(row_data)

    # Append the data for the current page to the list of all data
    all_data.extend(data)

    driver.maximize_window()
    # Find the "Next" link using the specified criteria
    #
    # Find the URL of the "Next" page, if available
    next_page_element = driver.find_element(By.XPATH, "//li[@class='page-item' and not(@aria-disabled='true')]/a[@class='page-link' and contains(@aria-label, 'Next »')]")
    next_page_url = next_page_element.get_attribute('href')

    if next_page_url:
        # Navigate to the next page
        driver.get(next_page_url)
    else:
        break  # No more pages to scrape

# Create a DataFrame from all the extracted data
df = pd.DataFrame(all_data[1:], columns=all_data[0])

# Close the Selenium web driver
driver.quit()

with open('result.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(all_data)

# Display the extracted data in a DataFrame
print(df)


       Pos   球員    排名      勝率    直板勝率    橫板勝率    右手勝率    左手勝率    男子勝率    女子勝率  \
0        1  葉柏旻     1   66.67  100.00   62.50   75.00    0.00   66.67       -   
1        2  黃建都     2  100.00       -  100.00  100.00  100.00  100.00  100.00   
2        3  林晉霆     3  100.00       -  100.00  100.00       -  100.00  100.00   
3        4  洪敬愷     4  100.00       -  100.00  100.00  100.00  100.00       -   
4        5  王志耿     5   88.00   85.71   88.37   90.00   80.00   87.50  100.00   
...    ...  ...   ...     ...     ...     ...     ...     ...     ...     ...   
2875  2876  徐聲泰  2870    0.00       -    0.00    0.00    0.00    0.00       -   
2876  2877  蔡傑穎  2871    0.00       -    0.00    0.00    0.00    0.00    0.00   
2877  2878  劉晨恩  2872    0.00       -    0.00    0.00    0.00    0.00    0.00   
2878  2879  龔育賢  2873    0.00       -    0.00    0.00       -    0.00       -   
2879  2880  劉靖恩  2874    0.00    0.00    0.00    0.00    0.00    0.00    0.00   

         穩定率    爆發率    平面勝率